In [1]:
import os
import pandas as pd
import psycopg2
import numpy as np

import warnings
warnings.filterwarnings('ignore')

conn = psycopg2.connect(
database="postgres", user='ad_postgres', password='AWSw0rksh0p', 
host='db-anomalydetect-postgres.chanowujpkf4.us-east-1.rds.amazonaws.com')
GET_ALL_ROWS = 'Select * from newkaggle order by timestamp'
try:
    with conn:
        original_df = pd.read_sql_query(GET_ALL_ROWS, conn)
        original_df.set_index('timestamp', inplace=True)
except (Exception, psycopg2.DatabaseError) as err:
    print(err)
finally:
    conn.close()

In [2]:
df =  original_df.copy()

nulls_series = df.isnull().sum()
print(nulls_series.sort_values())

machine_status        0
sensor_15             0
sensor_20            16
sensor_19            16
sensor_24            16
sensor_27            16
sensor_28            16
sensor_31            16
sensor_21            16
sensor_23            16
sensor_34            16
sensor_35            16
sensor_36            16
sensor_37            16
sensor_33            16
sensor_10            19
sensor_12            19
sensor_13            19
sensor_05            19
sensor_04            19
sensor_03            19
sensor_02            19
sensor_11            19
sensor_26            20
sensor_14            21
sensor_38            27
sensor_49            27
sensor_48            27
sensor_47            27
sensor_46            27
sensor_45            27
sensor_44            27
sensor_43            27
sensor_42            27
sensor_41            27
sensor_40            27
sensor_39            27
sensor_16            31
sensor_25            36
sensor_22            41
sensor_18            46
sensor_17       

In [3]:
empty_cols = nulls_series.sort_values().tail(3)
display(empty_cols)
bad_col_list = list(empty_cols.keys())

df.drop(bad_col_list, axis=1, errors='ignore', inplace=True)
print(df.columns)

sensor_00    10208
sensor_51    15383
sensor_50    77017
dtype: int64

Index(['sensor_01', 'sensor_02', 'sensor_03', 'sensor_04', 'sensor_05',
       'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09', 'sensor_10',
       'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15',
       'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20',
       'sensor_21', 'sensor_22', 'sensor_23', 'sensor_24', 'sensor_25',
       'sensor_26', 'sensor_27', 'sensor_28', 'sensor_29', 'sensor_30',
       'sensor_31', 'sensor_32', 'sensor_33', 'sensor_34', 'sensor_35',
       'sensor_36', 'sensor_37', 'sensor_38', 'sensor_39', 'sensor_40',
       'sensor_41', 'sensor_42', 'sensor_43', 'sensor_44', 'sensor_45',
       'sensor_46', 'sensor_47', 'sensor_48', 'sensor_49', 'machine_status'],
      dtype='object')


In [4]:
status_map = {'NORMAL': 0, 'BROKEN': 1, 'RECOVERING':0.5}
df['machine_status'] = df['machine_status'].map(status_map)

In [ ]:
col_averages = df.mean()
print(col_averages)
df.fillna(value=col_averages, inplace=True)

In [6]:
df['sensor_15'].unique()

array([0.])

In [7]:
df.drop('sensor_15', axis=1, errors='ignore', inplace=True)

In [8]:
df.to_csv('clean-df.csv')